In [1]:
X_sentence=list()
Y_sentence=list()
count=0
base_path="dataset"
X_train=list()
Y_train=list()
X_val=list()
Y_val=list()
X_test=list()
Y_test=list()
with open(base_path+"/S1-test.txt", encoding="utf8") as file:
    for line in file:
        if(line!='\n'):
            X_sentence.append(line.split(" ")[0])
            Y_sentence.append(line.split(" ")[1][:-1])
        else:
            count+=1
            if(count==2):
                X_test.append(X_sentence)      
                Y_test.append(Y_sentence)        
                X_sentence=list()
                Y_sentence=list()
                count=0
with open(base_path+"/S2-train.txt", encoding="utf8") as file:
    for line in file:
        if(line!='\n'):
            X_sentence.append(line.split(" ")[0])
            Y_sentence.append(line.split(" ")[1][:-1])
        else:
            count+=1
            if(count==2):
                X_train.append(X_sentence)      
                Y_train.append(Y_sentence)        
                X_sentence=list()
                Y_sentence=list()
                count=0
with open(base_path+"/S3-val.txt", encoding="utf8") as file:
    for line in file:
        if(line!='\n'):
            X_sentence.append(line.split(" ")[0])
            Y_sentence.append(line.split(" ")[1][:-1])
        else:
            count+=1
            if(count==2):
                X_val.append(X_sentence)      
                Y_val.append(Y_sentence)        
                X_sentence=list()
                Y_sentence=list()
                count=0
# sentences it is a list of sentences. Every sentence is a list of tuple
tokenized_sentences=X_train+X_test+X_val
tokenized_labels=Y_train+Y_test+Y_val

In [2]:
from gensim.models import FastText

model_2=FastText(sentences=tokenized_sentences, vector_size=100, window=10, min_count=5, sg=1, negative=10, alpha=0.01, sample=0.0001)

In [3]:
model_2.train(tokenized_sentences, total_examples=len(tokenized_sentences), epochs=2)

(304730, 699104)

In [4]:
import importlib
from modelli import wrapper, utils
importlib.reload(wrapper)
from modelli.wrapper import Sequence


lstm=Sequence(embeddings=model_2.wv, initial_vocab=model_2.wv.key_to_index)
lstm.fit_vocab(X_train,Y_train)

2023-03-02 17:52:50.488977: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-02 17:52:54.147351: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-02 17:52:54.148145: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-02 17:53:03.082454: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [ ]:
from modelli.wrapper import Sequence
import pickle

lstm=Sequence.load("param","weights.h5","preprocessor")
lstm.model.compile( optimizer=lstm.optimizer)
with open('optimizer.pkl', 'rb') as f:
    weight_values = pickle.load(f)
lstm.model.optimizer.set_weights(weight_values)


In [5]:
import tensorflow as tf
lstm.model.call((tf.keras.Input(shape=[47]),tf.keras.Input(shape=[47,23])))
# 47 = lenght of longer sequence
# 23 = lenght of longher word

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


<KerasTensor: shape=(None, 47, 49) dtype=float32 (created by layer 'crf')>

In [6]:
import importlib
import modelli.trainer
importlib.reload(modelli.trainer)
from modelli.trainer import Trainer



trainer = Trainer(lstm.model, preprocessor=lstm.p)
trainer.train(X_train, Y_train, X_val, Y_val, epochs=1, batch_size=32,verbose=1,shuffle=True)


  1/295 [..............................] - ETA: 7:01:50

: 

: 

In [ ]:
lstm.model.compile(loss=lstm.model.loss, optimizer='adam')

In [9]:
pred=lstm.model((tf.zeros(shape=[32,47]),tf.zeros(shape=[32,47,23])))
true_val=tf.zeros(shape=(1,47,49))
loss=lstm.model.compute_loss(x=None,y=true_val,y_pred=pred,sample_weight=None)
print(loss)

call
add_boundary
forward
recursion
loss
add_boundary
get_energy
forward
recursion
get_negative
tf.Tensor(
[3.9022448 3.9022448 3.9022448 3.9022448 3.9022448 3.9022448 3.9022448
 3.9022448 3.9022448 3.9022448 3.9022448 3.9022448 3.9022448 3.9022448
 3.9022448 3.9022448 3.9022448 3.9022448 3.9022448 3.9022448 3.9022448
 3.9022448 3.9022448 3.9022448 3.9022448 3.9022448 3.9022448 3.9022448
 3.9022448 3.9022448 3.9022448 3.9022448], shape=(32,), dtype=float32)
tf.Tensor(
[3.9022448 3.9022448 3.9022448 3.9022448 3.9022448 3.9022448 3.9022448
 3.9022448 3.9022448 3.9022448 3.9022448 3.9022448 3.9022448 3.9022448
 3.9022448 3.9022448 3.9022448 3.9022448 3.9022448 3.9022448 3.9022448
 3.9022448 3.9022448 3.9022448 3.9022448 3.9022448 3.9022448 3.9022448
 3.9022448 3.9022448 3.9022448 3.9022448], shape=(32,), dtype=float32)


In [ ]:
from keras.backend import batch_get_value
import pickle

lstm.save("param","weights.h5","preprocessor")
symbolic_weights = getattr(lstm.model.optimizer, 'weights')
weight_values = batch_get_value(symbolic_weights)
with open('optimizer.pkl', 'wb') as f:
    pickle.dump(weight_values, f)
